### Individual Income Prediction Using Random Forest

#### The most powerful tool for reducing decision tree overfitting is called the random forest algorithm. In this mission, we'll learn how to construct and apply random forests.

#### We'll use the same 1994 census data set on U.S. income. It contains information on marital status, age, type of work, and more. The target column, high_income, records salaries less than or equal to 50k a year (0), and more than 50k a year (1).

#### A random forest is a kind of ensemble model. Ensembles combine the predictions of multiple models to create a more accurate final prediction. We'll make a simple ensemble to see how they work.

#### Let's create two decision trees with slightly different parameters:

   - One with min_samples_leaf set to 2
   - One with max_depth set to 5

#### Then, we'll check their accuracies separately. Next, we'll combine their predictions and compare the combined accuracy with the individual accuracies of both trees.

In [1]:
# import libraries and data
import pandas
# Set index_col to False to avoid pandas thinking that the first column is row indexes (it's age)
income = pandas.read_csv("income.csv", index_col=False)
income.head(5)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,high_income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


### Clean the data

In [2]:
marital_mapper = {' Married-civ-spouse': 0,
                 ' Never-married': 1,
                 ' Divorced': 2,
                 ' Separated': 3,
                 ' Widowed': 4,
                 ' Married-spouse-absent': 5,
                 ' Married-AF-spouse': 6}
income['marital_status'] = income['marital_status'].map(marital_mapper)

In [3]:
workclass_mapper = {' Private': 0,
                   ' Self-emp-not-inc': 1,
                   ' Local-gov': 2,
                   ' ?': 3,
                   ' State-gov': 4,
                   ' Self-emp-inc': 5,
                   ' Federal-gov': 6,
                   ' Without-pay': 7,
                   ' Never-worked': 8}
income['workclass'] = income['workclass'].map(workclass_mapper)

In [4]:
occupation_mapper = {' Prof-specialty': 0,
                     ' Craft-repair': 1,     
                     ' Exec-managerial': 2,  
                     ' Adm-clerical': 3,     
                     ' Sales': 4,            
                     ' Other-service':5,    
                     ' Machine-op-inspct':6,
                     ' ?': 7,                
                     ' Transport-moving': 8, 
                     ' Handlers-cleaners': 9,
                     ' Farming-fishing': 10,  
                     ' Tech-support': 11,     
                     ' Protective-serv': 12,  
                     ' Priv-house-serv': 13,  
                     ' Armed-Forces': 14}
income['occupation'] = income['occupation'].map(occupation_mapper)

In [5]:
relationship_mapper = { ' Husband': 0,
                         ' Not-in-family': 1,
                         ' Own-child': 2,
                         ' Unmarried': 3,
                         ' Wife': 4,
                         ' Other-relative': 5}
income['relationship'] = income['relationship'].map(relationship_mapper)

In [6]:
race_mapper = {  ' White': 0,
                 ' Black': 1,
                 ' Asian-Pac-Islander': 2,
                 ' Amer-Indian-Eskimo': 3,
                 ' Other'             : 4}
income['race'] = income['race'].map(race_mapper)

In [7]:
sex_mapper = {' Male': 0, ' Female': 1}
income['sex'] = income['sex'].map(sex_mapper)

In [8]:
high_income_mapper = {' >50K': 1, ' <=50K': 0}
income['high_income'] = income['high_income'].map(high_income_mapper)

In [9]:
native_country_counts = income['native_country'].value_counts()
native_country_mapper = {}
for i, name in enumerate(native_country_counts.index):
    native_country_mapper[name] = i
print(native_country_mapper)
income['native_country'] = income['native_country'].map(native_country_mapper)

{' United-States': 0, ' Mexico': 1, ' ?': 2, ' Philippines': 3, ' Germany': 4, ' Canada': 5, ' Puerto-Rico': 6, ' El-Salvador': 7, ' India': 8, ' Cuba': 9, ' England': 10, ' Jamaica': 11, ' South': 12, ' China': 13, ' Italy': 14, ' Dominican-Republic': 15, ' Vietnam': 16, ' Guatemala': 17, ' Japan': 18, ' Poland': 19, ' Columbia': 20, ' Taiwan': 21, ' Haiti': 22, ' Iran': 23, ' Portugal': 24, ' Nicaragua': 25, ' Peru': 26, ' Greece': 27, ' France': 28, ' Ecuador': 29, ' Ireland': 30, ' Hong': 31, ' Trinadad&Tobago': 32, ' Cambodia': 33, ' Laos': 34, ' Thailand': 35, ' Yugoslavia': 36, ' Outlying-US(Guam-USVI-etc)': 37, ' Hungary': 38, ' Honduras': 39, ' Scotland': 40, ' Holand-Netherlands': 41}


In [10]:
income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         32561 non-null int64
fnlwgt            32561 non-null int64
education         32561 non-null object
education_num     32561 non-null int64
marital_status    32561 non-null int64
occupation        32561 non-null int64
relationship      32561 non-null int64
race              32561 non-null int64
sex               32561 non-null int64
capital_gain      32561 non-null int64
capital_loss      32561 non-null int64
hours_per_week    32561 non-null int64
native_country    32561 non-null int64
high_income       32561 non-null int64
dtypes: int64(14), object(1)
memory usage: 3.7+ MB


In [16]:
split = int(income.shape[0]* 0.8)
train = income.iloc[:split]
test = income.iloc[split:]
print(income.shape[0] == train.shape[0] + test.shape[0])

True


In [17]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26048 entries, 0 to 26047
Data columns (total 15 columns):
age               26048 non-null int64
workclass         26048 non-null int64
fnlwgt            26048 non-null int64
education         26048 non-null object
education_num     26048 non-null int64
marital_status    26048 non-null int64
occupation        26048 non-null int64
relationship      26048 non-null int64
race              26048 non-null int64
sex               26048 non-null int64
capital_gain      26048 non-null int64
capital_loss      26048 non-null int64
hours_per_week    26048 non-null int64
native_country    26048 non-null int64
high_income       26048 non-null int64
dtypes: int64(14), object(1)
memory usage: 3.0+ MB


In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]
clf = DecisionTreeClassifier(random_state=1, min_samples_leaf=2)
clf.fit(train[columns], train["high_income"])

clf2 = DecisionTreeClassifier(random_state=1, max_depth=5)
clf2.fit(train[columns], train["high_income"])
predictions = clf.predict(test[columns])
print(roc_auc_score(test["high_income"], predictions))

predictions = clf2.predict(test[columns])
print(roc_auc_score(test["high_income"], predictions))

0.6863009490128231
0.6848925045796866


#### When we have multiple classifiers making predictions, we can treat each set of predictions as a column in a matrix. Here's an example where we have Decision Tree 1 (DT1), Decision Tree 2 (DT2), and Decision Tree 3 (DT3):

DT1     DT2    DT3

0       1      0

1       1      1

0       0      1

1       0      0

#### Whenever we add more models to our ensemble, we just add more columns to the combined predictions. Ultimately, we don't want this matrix, though -- we want one prediction per row in the training data. To accomplish this, we'll need to create rules to convert each row of our matrix of predictions into a single number.

#### We want to create a Final Prediction vector that looks like this:

DT1     DT2    DT3    Final Prediction

0       1      0      0

1       1      1      1

0       0      1      0

1       0      0      0

#### There are many ways to get from the output of multiple models to a final vector of predictions. One method is majority voting, in which each classifier gets a "vote," and the most commonly voted value for each row "wins." This only works if there are more than two classifiers (and ideally an odd number, so we don't have to write a rule to break ties). Majority voting is what we applied in the example above.

#### Because we only had two classifiers, we'll have to use a different method to combine predictions. We'll take the mean of all of the items in a row. Right now, we're using the predict() method, which returns either 0 or 1. predict() returns something like this:

0

1

0

1

#### We can use the RandomForestClassifier.predict_proba() method instead, which will predict a probability from 0 to 1 that a given class is the right one for a row. Because 0 and 1 are our two classes, we'll get a matrix containing the number of rows in the income dataframe, and two columns. predict_proba() will return a result that looks like this:

0     1

.7    .3

.2    .8

.1    .9

#### Each row will correspond to a prediction. The first column is the probability that the prediction is a 0, and the second column is the probability that the prediction is a 1. Each row adds up to 1.

#### If we just take the second column, we get the average value that the classifier would predict for that row. If there's a .9 probability that the correct classification is 1, we can use the .9 as the value the classifier is predicting. This will give us a continuous output in a single vector, instead of just 0 or 1.

#### Then we can add together all of the vectors we get through this method, and divide the sum by the total number of vectors to get the mean prediction made across the entire ensemble for a particular row. Finally, we round off to get a 0 or 1 prediction for the row.

#### If we use the predict_proba() method on both classifiers from the last screen to generate probabilities, take the mean for each row, and then round the results, we'll get ensemble predictions.

In [21]:
# Add predictions and predictions2, then divide by 2 to get the mean.
# Use numpy.round() to round all of the resulting predictions. 
import numpy as np
predictions = clf.predict_proba(test[columns])[:,1]
predictions2 = clf2.predict_proba(test[columns])[:,1]
combined = (predictions + predictions2) / 2
rounded = np.round(combined)
print(roc_auc_score(test["high_income"], rounded))

0.7229448020557704


#### We can see from the previous screen, the combined predictions of the two trees had a higher AUC than that of either tree on its own.

#### The more "diverse" or dissimilar the models we use to construct an ensemble are, the stronger their combined predictions will be (assuming that all of the models have about the same accuracy). Ensembling a decision tree and a logistic regression model, for example, will result in stronger predictions than ensembling two decision trees with similar parameters. That's because those two models use very different approaches to arrive at their answers.

#### On the other hand, if the models we ensemble are very similar in how they make predictions, ensembling will result in a negligible boost.

#### Ensembling models with very different accuracies generally won't improve overall accuracy. Ensembling a model with a .75 AUC and a model with a .85 AUC on a test set will usually result in an AUC somewhere in between the two original values.

#### A random forest is an ensemble of decision trees. If we don't make any modifications to the trees, each tree will be exactly the same, so we'll get no boost when we ensemble them. In order to make ensembling effective, we have to introduce variation into each individual decision tree model.

#### If we introduce variation, each tree will be be constructed slightly differently, and will therefore make different predictions. This variation is what puts the "random" in "random forest."

#### There are two main ways to introduce variation in a random forest -- bagging and random feature subsets.

#### In a random forest, we don't train each tree on the entire data set. We train it on a random sample of the data, or a "bag," instead. We perform this sampling with replacement, which means that after we select a row from the data we're sampling, we put the row back in the data so it can be picked again. Some rows from the original data may appear in the "bag" multiple times.

### Bagging

In [23]:
# We'll build 10 trees
tree_count = 10

# Each "bag" will have 60% of the number of original rows
bag_proportion = .6

predictions = []
for i in range(tree_count):
    # We select 60% of the rows from train, sampling with replacement
    # We set a random state to ensure we'll be able to replicate our results
    # We set it to i instead of a fixed value so we don't get the same sample in every loop
    # That would make all of our trees the same
    bag = train.sample(frac=bag_proportion, replace=True, random_state=i)
    
    # Fit a decision tree model to the "bag"
    clf = DecisionTreeClassifier(random_state=1, min_samples_leaf=2)
    clf.fit(bag[columns], bag["high_income"])
    
    # Using the model, make predictions on the test data
    predictions.append(clf.predict_proba(test[columns])[:,1])
combined = np.sum(predictions, axis=0) / 10
rounded = np.round(combined)

print(roc_auc_score(test["high_income"], rounded))

0.7450233436800325


#### Using the bagging, we gained some accuracy over a single decision tree. To be exact, we achieved an AUC score of around .745 with bagging, which is an improvement over the AUC score we got without bagging.

#### Let's go back to the decision tree algorithm to explain random feature subsets:

   - First we pick the maximum number of features we want to evaluate each time we split the tree.
       - This has to be less than the total number of columns in the data.
   - Every time we split, we pick a random sample of features from the data.
   - Then we compute the information gain for each feature in our random sample, and pick the one with the highest information gain to split on.

#### To understand how splits work, let's look at information gain or entropy. Entropy is the measure of "disorder" in the data set. If a dataset has all the same labels, they'll have low entropy. If all the labels are different, they'll have high entropy. Splits that give us more information about the data, will ideally minimize entropy. In other words, the tree will ideally split the labels into distinct groups with as little mixture possible. This'll allow the splits to give our tree more predictive power.

#### We're repeating the same process to select the optimal split that minimizes entropy for a node. However, we'll only evaluate a constrained set of features that we select randomly. This introduces variation into the trees, and makes for more powerful ensembles.

### Random Feature Subsets

In [30]:
import math
# Create the data set that we used two missions ago
data = pandas.DataFrame([
    [0,4,20,0],
    [0,4,60,2],
    [0,5,40,1],
    [1,4,25,1],
    [1,5,35,2],
    [1,5,55,1]
    ])
data.columns = ["high_income", "employment", "age", "marital_status"]

# Set a random seed to make the results reproducible
np.random.seed(1)

# The dictionary to store our tree
tree = {}
nodes = []

def calc_entropy(column):
    """
    Calculate entropy given a pandas series, list, or numpy array.
    """
    # Compute the counts of each unique value in the column
    counts = np.bincount(column)
    # Divide by the total column length to get a probability
    probabilities = counts / len(column)
    
    # Initialize the entropy to 0
    entropy = 0
    # Loop through the probabilities, and add each one to the total entropy
    for prob in probabilities:
        if prob > 0:
            entropy += prob * math.log(prob, 2)
    
    return -entropy

def calc_information_gain(data, split_name, target_name):
    """
    Calculate information gain given a data set, column to split on, and target
    """
    # Calculate the original entropy
    original_entropy = calc_entropy(data[target_name])
    
    # Find the median of the column we're splitting
    column = data[split_name]
    median = column.median()
    
    # Make two subsets of the data, based on the median
    left_split = data[column <= median]
    right_split = data[column > median]
    
    # Loop through the splits and calculate the subset entropies
    to_subtract = 0
    for subset in [left_split, right_split]:
        prob = (subset.shape[0] / data.shape[0]) 
        to_subtract += prob * calc_entropy(subset[target_name])
    
    # Return information gain
    return original_entropy - to_subtract

# The function to find the column to split on
def find_best_column(data, target_name, columns):
    information_gains = []
    
    # Insert your code here
    
    for col in columns:
        information_gain = calc_information_gain(data, col, "high_income")
        information_gains.append(information_gain)

    # Find the name of the column with the highest gain
    highest_gain_index = information_gains.index(max(information_gains))
    highest_gain = columns[highest_gain_index]
    return highest_gain

# The function to construct an ID3 decision tree
def id3(data, target, columns, tree):
    unique_targets = pandas.unique(data[target])
    nodes.append(len(nodes) + 1)
    tree["number"] = nodes[-1]

    if len(unique_targets) == 1:
        if 0 in unique_targets:
            tree["label"] = 0
        elif 1 in unique_targets:
            tree["label"] = 1
        return
    
    best_column = find_best_column(data, target, columns)
    column_median = data[best_column].median()
    
    tree["column"] = best_column
    tree["median"] = column_median
    
    left_split = data[data[best_column] <= column_median]
    right_split = data[data[best_column] > column_median]
    split_dict = [["left", left_split], ["right", right_split]]
    
    for name, split in split_dict:
        tree[name] = {}
        id3(split, target, columns, tree[name])


# Run the ID3 algorithm on our data set and print the resulting tree
id3(data, "high_income", ["employment", "age", "marital_status"], tree)
print(tree)
def find_best_column(data, target_name, columns):
    information_gains = []
    
    # Select two columns randomly
    cols = np.random.choice(columns, 2)
    
    for col in cols:
        information_gain = calc_information_gain(data, col, "high_income")
        information_gains.append(information_gain)

    highest_gain_index = information_gains.index(max(information_gains))
    
    # Get the highest gain by indexing "cols"
    highest_gain = cols[highest_gain_index]
    
    return highest_gain

id3(data, "high_income", ["employment", "age", "marital_status"], tree)
print(tree)

{'number': 1, 'column': 'employment', 'median': 4.5, 'left': {'number': 2, 'column': 'age', 'median': 25.0, 'left': {'number': 3, 'column': 'age', 'median': 22.5, 'left': {'number': 4, 'label': 0}, 'right': {'number': 5, 'label': 1}}, 'right': {'number': 6, 'label': 0}}, 'right': {'number': 7, 'column': 'age', 'median': 40.0, 'left': {'number': 8, 'column': 'age', 'median': 37.5, 'left': {'number': 9, 'label': 1}, 'right': {'number': 10, 'label': 0}}, 'right': {'number': 11, 'label': 1}}}
{'number': 12, 'column': 'age', 'median': 37.5, 'left': {'number': 13, 'column': 'employment', 'median': 4.0, 'left': {'number': 14, 'column': 'age', 'median': 22.5, 'left': {'number': 15, 'label': 0}, 'right': {'number': 16, 'label': 1}}, 'right': {'number': 17, 'label': 1}}, 'right': {'number': 18, 'column': 'age', 'median': 55.0, 'left': {'number': 19, 'column': 'age', 'median': 47.5, 'left': {'number': 20, 'label': 0}, 'right': {'number': 21, 'label': 1}}, 'right': {'number': 22, 'label': 0}}}


#### We can also repeat our random subset selection process in scikit-learn. We just set the splitter parameter on DecisionTreeClassifier to "random", and the max_features parameter to "auto". 

### Scikit-learn 

In [32]:
# We'll build 10 trees
tree_count = 10

# Each "bag" will have 60% of the number of original rows
bag_proportion = .6

predictions = []
for i in range(tree_count):
    # We select 60% of the rows from train, sampling with replacement
    # We set a random state to ensure we'll be able to replicate our results
    # We set it to i instead of a fixed value so we don't get the same sample every time
    bag = train.sample(frac=bag_proportion, replace=True, random_state=i)
    
    # Fit a decision tree model to the "bag"
    clf = DecisionTreeClassifier(random_state=1, min_samples_leaf=2)
    clf.fit(bag[columns], bag["high_income"])
    
    # Using the model, make predictions on the test data
    predictions.append(clf.predict_proba(test[columns])[:,1])

combined = np.sum(predictions, axis=0) / 10
rounded = np.round(combined)

print(roc_auc_score(test["high_income"], rounded))
predictions = []
for i in range(tree_count):
    # We select 60% of the rows from train, sampling with replacement
    # We set a random state to ensure we'll be able to replicate our results
    # We set it to i instead of a fixed value so we don't get the same sample every time
    bag = train.sample(frac=bag_proportion, replace=True, random_state=i)
    
    # Fit a decision tree model to the "bag"
    clf = DecisionTreeClassifier(random_state=1, min_samples_leaf=2, splitter="random", max_features="auto")
    clf.fit(bag[columns], bag["high_income"])
    
    # Using the model, make predictions on the test data
    predictions.append(clf.predict_proba(test[columns])[:,1])

combined = np.sum(predictions, axis=0) / 10
rounded = np.round(combined)

print(roc_auc_score(test["high_income"], rounded))

0.7450233436800325
0.7417773000203541


#### Using random subsets from the previous screen improved the accuracy over using bagging alone. 

#### We've demonstrated the two building blocks of random forests, bagging and random feature subsets. Luckily, we don't have to write code from scratch each time. Scikit-learn has a RandomForestClassifier class and a RandomForestRegressor class that enable us to train and test random forest models quickly.

#### When we instantiate a RandomForestClassifier, we pass in an n_estimators parameter that indicates how many trees to build. While adding more trees usually improves accuracy, it also increases the overall time the model takes to train.

#### RandomForestClassifier has a similar interface to DecisionTreeClassifier, and we can use the fit() and predict() methods to train and make predictions.

In [33]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=5, random_state=1, min_samples_leaf=2)
clf.fit(train[columns], train["high_income"])

predictions = clf.predict(test[columns])
print(roc_auc_score(test["high_income"], predictions))

0.748417718298392


#### Similar to decision trees, we can tweak some of the parameters for random forests, including:

   - min_samples_leaf
   - min_samples_split
   - max_depth
   - max_leaf_nodes

#### These parameters apply to the individual trees in the model, and change how they are constructed. There are also parameters specific to the random forest that alter its overall construction:

   - n_estimators
   - bootstrap - "Bootstrap aggregation" is another name for bagging; this parameter indicates whether to turn it on (Defaults to True)
   
#### Tweaking parameters can increase the accuracy of the forest. The easiest tweak is to increase the number of estimators we use. This approach yields diminishing returns -- going from 10 trees to 100 will make a bigger difference than going from 100 to 500, which will make a bigger difference than going from 500 to 1000. The accuracy increase function is logarithmic, so increasing the number of trees beyond a certain number (usually 200) won't help much at all.

In [34]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=5, random_state=1, min_samples_leaf=2)

clf.fit(train[columns], train["high_income"])

predictions = clf.predict(test[columns])
print(roc_auc_score(test["high_income"], predictions))
clf = RandomForestClassifier(n_estimators=150, random_state=1, min_samples_leaf=2)

clf.fit(train[columns], train["high_income"])

predictions = clf.predict(test[columns])
print(roc_auc_score(test["high_income"], predictions))

0.748417718298392
0.7499454254019947


#### We were able to improve the AUC, but the model using 150 trees took much longer to train. While the extra training time is trivial on the data set we're working with right now, understanding this trade-off will help you when you're working with much larger data sets where the extra time could amount to hours or days!

#### One of the major advantages of random forests over single decision trees is that they tend to overfit less. Although each individual decision tree in a random forest varies widely, the average of their predictions is less sensitive to the input data than a single tree is. This is because while one tree can construct an incorrect and overfit model, the average of 100 or more trees will be more likely to hone in on the signal and ignore the noise. The signal will be the same across all of the trees, whereas each tree will hone in on the noise differently. This means that the average will discard the noise and keep the signal.

In [35]:
clf = DecisionTreeClassifier(random_state=1, min_samples_leaf=5)

clf.fit(train[columns], train["high_income"])

predictions = clf.predict(train[columns])
print(roc_auc_score(train["high_income"], predictions))

predictions = clf.predict(test[columns])
print(roc_auc_score(test["high_income"], predictions))

clf = RandomForestClassifier(n_estimators=150, random_state=1, min_samples_leaf=5)
clf.fit(train[columns], train["high_income"])

predictions = clf.predict(train[columns])
print(roc_auc_score(train["high_income"], predictions))

predictions = clf.predict(test[columns])
print(roc_auc_score(test["high_income"], predictions))

0.8175182410598762
0.7282202193161003
0.7930348241883318
0.7547358411357623


#### We've fit a single decision tree to the training set, and made predictions for both the training and testing sets. The AUC for the training set predictions is .819, while the AUC for the testing set is .714. The fact that the test AUC is much lower than the train AUC indicates that the model is overfitting.

### Conclusion

#### While the random forest algorithm is incredibly powerful, it isn't applicable to all tasks. The main strengths of a random forest are:

   - Very accurate predictions - Random forests achieve near state-of-the-art performance on many machine learning tasks. Along with neural networks and gradient-boosted trees, they're typically one of the top-performing algorithms.
   - Resistance to overfitting - Due to their construction, random forests are fairly resistant to overfitting. We still need to set and tweak parameters like max_depth though.

#### The main weaknesses of using a random forest are:

   - They're difficult to interpret - Because we've averaging the results of many trees, it can be hard to figure out why a random forest is making predictions the way it is.
   - They take longer to create - Making two trees takes twice as long as making one, making three takes three times as long, and so on. Fortunately, we can exploit multicore processors to parallelize tree construction. Scikit allows us to do this through the n_jobs parameter on RandomForestClassifier. We'll discuss parallelization in greater detail later on.

#### Given these trade-offs, it makes sense to use random forests in situations where accuracy is of the utmost importance; being able to interpret or explain the decisions the model is making isn't key. In cases where time is of the essence or interpretability is important, a single decision tree may be a better choice.